# 评估
评估是检验语言模型问答质量的关键环节。评估可以检验语言模型在不同文档上的问答效果，找出其弱点。还可以通过比较不同模型，选择最佳系统。此外，定期评估也可以检查模型质量的衰减。评估通常有两个目的：

- 检验LLM应用是否达到了验收标准
- 分析改动对于LLM应用性能的影响
基本的思路就是利用语言模型本身和链本身，来辅助评估其他的语言模型、链和应用程序。

In [42]:
import os
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.evaluation.qa.generate_chain import QAGenerateChain 
from langchain.output_parsers.regex import RegexParser
from langchain.prompts import PromptTemplate
from langchain.base_language import BaseLanguageModel
from typing import Any
from langchain.evaluation.qa.eval_chain import QAEvalChain

# 创建语言模型
llm = ChatOpenAI(
    model="qwen-turbo",
    base_url=os.getenv("API_BASEURL"),
    api_key=os.getenv("API_KEY"),
    temperature = 0,
    streaming=True
)

# 加载数据
loader = TextLoader("product.txt")
data = loader.load()

# print("数据：",data)

# 文本切割
text_splitter = RecursiveCharacterTextSplitter(
    separators=["{"],
    chunk_size=150,
    chunk_overlap=10
)

documents = text_splitter.split_documents(data)

# for document in documents:
#     print(f"切割后的数据：{document.page_content}\n\n")

# 创建文本嵌入模型
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key=os.getenv("API_KEY"),
)

# 创建向量存储
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory="ChromaDB/Evaluate"
)

# 转换为检索器
retriever = vector_store.as_retriever();

# 检索链
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever
)

# 提供评估示例

examples = [
    {
        "query": "智能健康手表是什么颜色？",
        "answer": "深空黑。"
    },
    {
        "query": "笔记本电脑多大？",
        "answer": "14英寸。"
    }
]

template = """You are a teacher coming up with questions to ask on a quiz. 
Given the following document, please generate a question and answer based on that document.

Example Format:
<Begin Document>
...
<End Document>
QUESTION: question here
ANSWER: answer here

These questions should be detailed and be based explicitly on information in the document. Begin!

<Begin Document>
{doc}
<End Document>
请使用中文输出
"""

output_parser = RegexParser(
    regex=r"QUESTION: (.*?)\nANSWER: (.*)", output_keys=["query", "answer"]
)

PROMPT = PromptTemplate(
    input_variables=["doc"], template=template, output_parser=output_parser
)

# 继承QAGenerateChain
class ChineseQAGenerateChain(QAGenerateChain):
    """LLM Chain specifically for generating examples for question answering."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, **kwargs: Any) -> QAGenerateChain:
        """Load QA Generate Chain from LLM."""
        return cls(llm=llm, prompt=PROMPT, **kwargs)
    

example_gen_chain = ChineseQAGenerateChain.from_llm(llm)#通过传递chat open AI语言模型来创建这个链
new_examples = example_gen_chain.apply([{"doc": t} for t in documents]) 

# 将大模型生成的测试用例添加进测试集合里
examples += [ v for item in new_examples for k,v in item.items()]

for i in range(len(examples)):
    print(f"""
          示例答案：{i+1}
          query: {examples[i]['query']}
          answer: {examples[i]['answer']}
          """)

qa_chain.run(examples[0]["query"])

#为所有不同的示例创建预测
predictions = qa_chain.apply(examples)

#通过调用另一个大模型进行评估
eval_llm = ChatOpenAI(
    model="deepseek-r1",
    base_url=os.getenv("API_BASEURL"),
    api_key=os.getenv("API_KEY"),
    temperature = 0,
    streaming=True
)

eval_chain = QAEvalChain.from_llm(eval_llm)

# 评分输出信息
graded_outputs = eval_chain.evaluate(
    examples,
    predictions
)

for i, eg in enumerate(examples):
    print(f"Example {i+1}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print("="*20)




          示例答案：1
          query: 智能健康手表是什么颜色？
          answer: 深空黑。
          

          示例答案：2
          query: 笔记本电脑多大？
          answer: 14英寸。
          

          示例答案：3
          query: 智能健康手表Pro的标准版有哪些主要特点？
          answer: 智能健康手表Pro标准版的主要特点包括：1.4英寸的显示屏，颜色为深空黑；支持24小时心率监测和多种运动模式；并且具有长达14天的续航时间。
          

          示例答案：4
          query: 这款超轻便携笔记本电脑的主要规格是什么？
          answer: 这款笔记本电脑的主要规格是 i7 处理器、16GB 内存和 512GB 存储空间。
          

          示例答案：5
          query: 这款多功能厨房料理机的推荐理由是什么？
          answer: 推荐理由是“一机多用，搅拌、切碎、榨汁、研磨，厨房必备神器”。
          

          示例答案：6
          query: 这款纯棉休闲T恤的推荐理由是什么？
          answer: 推荐理由是其采用100%精梳棉面料，具有透气舒适的特性，并且是一款经典百搭的款式。
          

          示例答案：7
          query: 这款无线蓝牙耳机的推荐理由是什么？
          answer: 推荐理由包括主动降噪、高清音质、续航30小时，适合通勤和运动。
          

          示例答案：8
          query: 智能家居摄像头的推荐理由是什么？
          answer: 推荐理由包括360°全景监控、夜视功能、手机远程查看以及安全守护家庭。
          

          示例答案：9
          query: 防晒霜SPF50+的推荐理由是什么？
          answer: 推荐理由是：全面防护UV